In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

df = pd.read_csv("/content/mbti_2 (1).csv")


In [ ]:
df.describe()

In [ ]:
df['posts'][0]

In [ ]:
df['posts'][1]


In [ ]:
df['type'][1]

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys

In [ ]:
max_len = 256
train_batch_size = 32
valid_batch_size = 32
epochs = 2
learning_rate = 5e-05


In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
class Custom_dataset(torch.utils.data.Dataset):
  def __init__(self,df,tokenizer,max_len):
    self.df = df
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.title = self.df['posts']
    self.targets = df[target_columns].values

  def __len__(self):
    return len(self.title)

  def __getitem__(self,index):
    title = str(self.title[index])
    title = ' '.join(title.split())

    inputs = self.tokenizer.encode_plus(
          title,
          None,
          add_special_tokens = True,
          max_length = self.max_len,
          padding = 'max_length',
          return_token_type_ids = True,
          truncation = True,
          return_attention_mask = True,
          return_tensors = 'pt'
          )
    target = self.targets[index]

        # Если пришло 16 чисел, берем индекс максимального (где стоит 1)
    if len(target.shape) > 0 and target.size > 1: # проверяет если это список как [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] и его длина большем чем 1 то np.argmax
                                                  # возвращает позицию самого большого, в нашем случае единицы
      target = np.argmax(target)
    return {
          'input_ids': inputs['input_ids'].flatten(),
          'attention_mask':  inputs['attention_mask'].flatten(),
          'token_type_ids':  inputs['token_type_ids'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
      }

In [ ]:
df[target_columns].values

In [ ]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[['type']])


encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['type']))


df = pd.concat([df, encoded_df], axis=1)


In [ ]:
df['type'].unique()

In [ ]:
target_columns = ['type_INFJ', 'type_ENTP', 'type_INTP', 'type_INTJ', 'type_ENTJ', 'type_ENFJ', 'type_INFP', 'type_ENFP', 'type_ISFP', 'type_ISTP', 'type_ISFJ', 'type_ISTJ', 'type_ESTP', 'type_ESFP', 'type_ESTJ', 'type_ESFJ']
df

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=200)
train_df = train_df.reset_index(drop=True)
valid_df = val_df.reset_index(drop=True)

In [ ]:
train_data_set = Custom_dataset(train_df,tokenizer,max_len)
valid_data_set = Custom_dataset(val_df,tokenizer,max_len)

In [ ]:

batch = next(iter(train_data_set))


for key, value in batch.items():
    print(f"{key}: {value.shape}")

In [ ]:
train_data_loader  =  torch.utils.data.DataLoader(
    train_data_set,
    shuffle = True,
    batch_size = train_batch_size,
    num_workers = 0
)
valid_data_loader  =  torch.utils.data.DataLoader(
    train_data_set,
    shuffle = False,
    batch_size = valid_batch_size,
    num_workers = 0
)

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
  checkpoint = torch. load (checkpoint_fpath)
  model.load_state_dict(checkpoint['state _dict'])
  optimizer.load_state_dict(checkpoint['optimizer '])
  valid_loss_min = checkpoint ['valid_ loss_min']
  return model, optimizer, checkpoint ['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    torch.save(state, checkpoint_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class Bert_Class(nn.Module):
  def __init__(self):
    super(Bert_Class, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-uncased',return_dict = True)
    self.dropout = nn.Dropout(0.3)
    self.linear = nn.Linear(768,16)
  def forward(self,input_ids,attention_mask,token_type_ids):  #берем значит пуллеры размерностью 768 которые указывают характеристику каждого примера, через линейный слой
    output = self.bert_model(input_ids,attention_mask,token_type_ids) #выдаем логиты голосуя к какому классу относится пример, вычисляем лосс функция и повторяем
    output_dropout = self.dropout(output.pooler_output)
    output =  self.linear(output_dropout)
    return output
model = Bert_Class()
model.to(device)



In [ ]:

def loss_fn(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets)
optimizer = torch.optim.AdamW(params=model.parameters(), lr = learning_rate, eps=1e-8,weight_decay=0.01)

from transformers import get_linear_schedule_with_warmup

total_steps = len(train_data_loader) * 5
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, optimizer, checkpoint_path, best_model_path):
    valid_loss_min = np.inf

    for epoch in range(1, n_epochs + 1):
        train_loss = 0
        valid_loss = 0

        print(f'\\n--- Эпоха {epoch} из {n_epochs} ---')


        model.train()
        for index, batch in enumerate(training_loader):
            input_ids = batch['input_ids'].to(device, dtype=torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.long)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()


            if index % 10 == 0:
                print(f'Батч {index}: Текущий Loss = {loss.item():.4f}')


        model.eval()
        with torch.no_grad():
            for index, batch in enumerate(validation_loader):
                input_ids = batch['input_ids'].to(device, dtype=torch.long)
                attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
                token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
                targets = batch['targets'].to(device, dtype=torch.long)

                outputs = model(input_ids, attention_mask, token_type_ids)
                loss = loss_fn(outputs, targets)
                valid_loss += loss.item()


        train_loss = train_loss / len(training_loader)
        valid_loss = valid_loss / len(validation_loader)

        print(f'ИТОГ ЭПОХИ {epoch}: Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}')


        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        is_best = valid_loss <= valid_loss_min
        if is_best:
            print(f'!!! Найдена лучшая модель: Valid Loss снизился ({valid_loss_min:.4f} --> {valid_loss:.4f}). Сохраняю...')
            valid_loss_min = valid_loss

        save_ckp(checkpoint, is_best, checkpoint_path, best_model_path)


    print(f'\\nОбучение завершено. Загружаю лучшие веса из {best_model_path}')
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['state_dict'])

    return model

In [ ]:

EPOCHS = 10 # Начни с 3, чтобы проверить, как идет процесс
CHECKPOINT_PATH = "current_checkpoint.pt"
BEST_MODEL_PATH = "best_model.pt"

trained_model = train_model(
    n_epochs=EPOCHS,
    training_loader=train_data_loader ,
    validation_loader=valid_data_loader, # И этот тоже
    model=model,
    optimizer=optimizer,
    checkpoint_path=CHECKPOINT_PATH,
    best_model_path=BEST_MODEL_PATH
)

print("Обучение завершено!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def evaluate_model(data_loader, model, device):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch in data_loader:
            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids)
            _, preds = torch.max(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    print("\n" + "="*50)
    print("МЕТРИКИ НА ВАЛИДАЦИОННОЙ ВЫБОРКЕ")
    print("="*50)
    print(classification_report(all_targets, all_preds))

    plt.figure(figsize=(12, 10))
    sns.heatmap(confusion_matrix(all_targets, all_preds), annot=True, fmt='d', cmap='Blues')
    plt.title('Матрица ошибок (Confusion Matrix)')
    plt.xlabel('Предсказанный класс')
    plt.ylabel('Истинный класс')
    plt.show()


evaluate_model(valid_data_loader, model, device)